In [1]:
import numpy as np

X_train = np.load('./data_set/X_train.npy')
X_train.shape

(1000, 111)

In [2]:
X_train

array([[57.,  5.,  0., ...,  0.,  0.,  0.],
       [71.,  2.,  2., ...,  0.,  0.,  1.],
       [64.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [46.,  4.,  0., ...,  1.,  0.,  0.],
       [70.,  2.,  1., ...,  0.,  0.,  0.],
       [60.,  2.,  0., ...,  0.,  0.,  0.]])

1.计算每个特征的缺失值百分比
2.找出缺失值百分比较高的特征（>50%）
3.找出有一些缺失值的特征（0 < 缺失百分比 <= 50）
4.识别无缺失值的特征

In [3]:
# Calculate the percentage of missing values for each feature
missing_percentage = (np.isnan(X_train).sum(axis=0) / X_train.shape[0]) * 100

# Identify features with a high percentage of missing values (>50%)
features_high_missing = np.where(missing_percentage > 50)[0]

# Identify features with some missing values (0 < missing percentage <= 50)
features_some_missing = np.where((missing_percentage > 0) & (missing_percentage <= 50))[0]

# Identify features with no missing values
features_no_missing = np.where(missing_percentage == 0)[0]

# Summary
((len(features_high_missing), len(features_some_missing), len(features_no_missing)),
 missing_percentage[features_high_missing], missing_percentage[features_some_missing])


((4, 105, 2),
 array([96.3, 64.4, 64.4, 99.8]),
 array([ 0.5,  0.2,  6.7,  4.6,  3.4,  0.5,  0.4, 15.3,  3.4,  1.6,  1.6,
         1.6,  1.6,  1.6,  1.6,  1.6,  1.2,  1.2,  1.2,  1.2,  1.2,  1.2,
         1.2,  0.2,  0.2,  0.2,  0.3,  0.3,  0.3,  0.3,  0.3, 16.1, 16.1,
         0.7,  0.8,  0.9,  0.7,  0.7,  0.7,  5.1,  5. ,  4.8,  4.3,  9.6,
         9.6,  9.6,  9.6,  9.6,  9.6,  6.9,  6.9,  6.9,  6.9,  6.9,  6.9,
         6.9,  6.9,  6.9,  6.8,  6.8,  6.8,  6.8,  6.8,  6.8,  6.8,  6.8,
         6.8,  6.8,  6.8,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6, 20.1,
        20.1, 20.4, 20.4, 17.4, 17.5,  7.4, 11.5,  8. ,  0.9,  6.9,  8. ,
        37.1, 39.2, 38.2,  0.3,  0.3,  6.8,  8.2,  0.7,  6.9,  8.1,  0.5,
         0.4,  0.5,  0.8,  0.8,  0.7,  0.7]))

在`X_train`数据集中，我们发现特征的缺失情况如下：

- 有4个特征的缺失值超过了50%（分别是96.3%，64.4%，64.4%，和99.8%）。这些特征的缺失值非常高，可能需要考虑删除这些特征，因为它们可能不会提供足够的信息，或者填充这些缺失值可能会引入较大的偏差。
- 有105个特征的缺失值在0到50%之间，这表明大多数特征都有一定程度的缺失值。这些特征可以通过各种方法填充，例如使用众数、平均值、中位数或基于模型的预测方法。
- 有2个特征没有缺失值，这意味着这些特征已经是完整的，不需要进行缺失值处理。

根据不同特征的缺失值类型，我们可以采取以下步骤来处理缺失值：

1. **对于缺失值超过50%的特征**，考虑到缺失值非常多，可以选择删除这些特征，或者如果有足够的理由相信这些特征对预测非常重要，可以尝试使用更复杂的填充方法，如基于模型的预测。

2. **对于缺失值在0到50%的特征**，可以根据特征的性质（连续还是分类）选择适当的填充方法。例如，对于连续型特征，可以使用平均值或中位数填充；对于分类特征，可以使用众数填充。对于这些特征，也可以考虑使用K最近邻（KNN）或其他基于模型的方法来填充缺失值。

3. **没有缺失值的特征**不需要进行缺失值处理。

判断数据集特征为离散还是连续通常基于特征的性质和上下文知识。在实践中，可以根据以下几点来辨识：

1. **值的类型**：连续特征通常可以取任何值在一个范围内，包括小数；而离散特征的值通常是整数或有限的类别。

2. **值的数量**：如果一个特征只有少量不同的值，尤其是如果这些值表示类别，那么这个特征可能是离散的。相反，如果一个特征有很多不同的值，那么它可能是连续的。

3. **上下文知识**：有时候需要根据特征的实际含义来判断。例如，人的年龄虽然通常用整数表示，但由于它可以在较大范围内变化，所以被视为连续特征。

在没有明确信息区分特征为连续还是离散的情况下，一种方法是基于不同值的数量来推断：如果一个特征的唯一值数量占样本总数的一个较小比例，且特征值看起来像是类别标签（例如整数序列），那么可以假设这个特征是离散的。

接下来，编写预处理函数，以便根据特征的性质（假设基于不同值的数量）自动选择填充策略：

- 对于假定的连续特征（基于不同值的数量占比较高），我们将使用均值填充。
- 对于假定的离散特征（不同值较少），我们将使用众数填充。

让我们尝试实现这种策略。

数据已经根据特征的离散或连续性质进行了预处理，并且根据缺失值比例的不同阈值（90%和50%）保存了两个版本的数据集：

1. 删除超过90%缺失值的特征，并根据特征类型应用了不同的填充策略（均值填充连续特征，众数填充离散特征）。处理后的数据集保存为`X_train_preprocessed_type_90.npy`，包含1000个观测值和109个特征。

2. 删除超过50%缺失值的特征，并同样根据特征类型应用了不同的填充策略。处理后的数据集保存为`X_train_preprocessed_type_50.npy`，包含1000个观测值和107个特征。

这种处理方式提供了一种基于特征属性自动选择填充策略的方法，有助于更精细地处理不同类型的缺失数据。

In [8]:
from sklearn.impute import SimpleImputer


def preprocess_data_with_type_detection(X, missing_threshold):
    # Calculate the percentage of missing values for each feature
    missing_percentage = (np.isnan(X).sum(axis=0) / X.shape[0]) * 100
    # Identify features to drop based on the missing_threshold
    features_to_drop = np.where(missing_percentage > missing_threshold)[0]
    X_reduced = np.delete(X, features_to_drop, axis=1)

    # Detect discrete and continuous features based on the number of unique values
    # Here we define a threshold (e.g., 5% of the total observations) to distinguish between the two
    unique_threshold = 0.05 * X_reduced.shape[0]
    discrete_features = [i for i in range(X_reduced.shape[1]) if
                         len(np.unique(np.nan_to_num(X_reduced[:, i]))) <= unique_threshold]
    continuous_features = [i for i in range(X_reduced.shape[1]) if i not in discrete_features]

    # Apply appropriate imputation strategies
    # Continuous features: mean imputation
    continuous_imputer = SimpleImputer(strategy='mean')
    X_reduced[:, continuous_features] = continuous_imputer.fit_transform(X_reduced[:, continuous_features])

    # Discrete features: most frequent (mode) imputation
    discrete_imputer = SimpleImputer(strategy='most_frequent')
    X_reduced[:, discrete_features] = discrete_imputer.fit_transform(X_reduced[:, discrete_features])

    return X_reduced


# Preprocess the data with type detection and different missing thresholds
X_train_preprocessed_type_90 = preprocess_data_with_type_detection(X_train, 90)
X_train_preprocessed_type_50 = preprocess_data_with_type_detection(X_train, 50)

# Save the preprocessed datasets with type detection
np.save('./processed_data_set/X_train_preprocessed_type_90.npy', X_train_preprocessed_type_90)
np.save('./processed_data_set/X_train_preprocessed_type_50.npy', X_train_preprocessed_type_50)

X_train_preprocessed_type_90.shape, X_train_preprocessed_type_50.shape


((1000, 111), (1000, 111))

In [10]:
X_train_preprocessed_type_90

array([[57.,  5.,  0., ...,  0.,  0.,  0.],
       [71.,  2.,  2., ...,  0.,  0.,  1.],
       [64.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [46.,  4.,  0., ...,  1.,  0.,  0.],
       [70.,  2.,  1., ...,  0.,  0.,  0.],
       [60.,  2.,  0., ...,  0.,  0.,  0.]])

In [11]:
X_train_preprocessed_type_90

array([[57.,  5.,  0., ...,  0.,  0.,  0.],
       [71.,  2.,  2., ...,  0.,  0.,  1.],
       [64.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [46.,  4.,  0., ...,  1.,  0.,  0.],
       [70.,  2.,  1., ...,  0.,  0.,  0.],
       [60.,  2.,  0., ...,  0.,  0.,  0.]])

统计哪些特征是离散的，哪些是连续的

In [13]:
def identify_feature_types(X, unique_threshold_ratio=0.05):
    """
    Identify discrete and continuous features in a dataset.

    Parameters:
    - X: numpy array, the dataset to analyze.
    - unique_threshold_ratio: float, the threshold ratio to distinguish between discrete and continuous features.
      A feature is considered discrete if the number of unique values is less than or equal to this ratio times
      the number of observations.

    Returns:
    - discrete_features: list of column indices that are identified as discrete features.
    - continuous_features: list of column indices that are identified as continuous features.
    """
    # Initialize lists to store the indices of discrete and continuous features
    discrete_features = []
    continuous_features = []

    # Calculate the unique threshold based on the ratio and the number of observations
    unique_threshold = unique_threshold_ratio * X.shape[0]

    # Iterate through each feature to determine its type
    for i in range(X.shape[1]):
        # Count the number of unique values, ignoring NaN values
        unique_values = len(np.unique(np.nan_to_num(X[:, i], nan=np.nan)))

        # Determine if the feature is discrete or continuous based on the unique threshold
        if unique_values <= unique_threshold:
            discrete_features.append(i)
        else:
            continuous_features.append(i)

    return discrete_features, continuous_features


# Use the function to identify feature types in the original dataset
discrete_features, continuous_features = identify_feature_types(X_train)

# discrete_features, continuous_features
print(f"Discrete features: {discrete_features}")
print(f"Continuous features: {continuous_features}")

Discrete features: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 87, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]
Continuous features: [0, 34, 35, 36, 83, 85, 86, 88, 89, 90]
